<a href="https://colab.research.google.com/github/Henriqueue/heart-disease-prediction/blob/main/Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Pré-processamento e Métricas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # Para normalizar os dados
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Para o Pipeline
from sklearn.pipeline import Pipeline

# Configuração para plots
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Carregar a base de dados (assumindo que você fez o upload do 'heart.csv')
df = pd.read_csv('heart.csv')

# Vamos dar uma olhada rápida nos dados
print(df.info())
print("\nPrimeiras linhas do dataset:")
print(df.head())

In [ ]:
# Definindo o estilo para os gráficos de EDA
sns.set_style("whitegrid")

# Célula para Gráficos de EDA

# 1. Qual é o balanceamento da nossa variável alvo?
plt.figure(figsize=(6, 4))
sns.countplot(x='target', data=df)
plt.title('Distribuição da Doença Cardíaca (0=Não, 1=Sim)')
plt.xlabel('Condição')
plt.ylabel('Contagem')
plt.savefig('target_distribution.png') # Salvar para o README
plt.show()

# 2. Como os atributos numéricos se correlacionam?
# Este é o GRÁFICO MAIS IMPORTANTE para "relação entre atributos"
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlação')
plt.savefig('correlation_heatmap.png') # Salvar para o README
plt.show()

# 3. Como a idade afeta a chance de ter doença?
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='age', hue='target', kde=True, bins=30)
plt.title('Distribuição de Idade por Condição Cardíaca')
plt.xlabel('Idade')
plt.ylabel('Contagem')
plt.legend(title='Condição', labels=['1: Doente', '0: Saudável'])
plt.savefig('age_distribution.png') # Salvar para o README
plt.show()

# 4. (Bônus) Pessoas com dor no peito (cp) induzida por exercício (exang)
#    têm maior frequência cardíaca máxima (thalach)?
plt.figure(figsize=(10, 6))
sns.boxplot(x='exang', y='thalach', data=df, hue='target')
plt.title('Freq. Cardíaca Máx. vs. Angina Induzida por Exercício')
plt.xlabel('Angina Induzida por Exercício (0=Não, 1=Sim)')
plt.ylabel('Freq. Cardíaca Máxima (thalach)')
plt.show()

In [ ]:
# 1. Separando features (X) e alvo (y)
# X são todas as colunas, exceto 'target'
X = df.drop('target', axis=1)
# y é apenas a coluna 'target'
y = df['target']

# 2. Dividir a base em Treino e Teste
# Usaremos 70% para treino e 30% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Tamanho de X_train: {X_train.shape}")
print(f"Tamanho de X_test: {X_test.shape}")

In [ ]:
# 3. Normalizar os dados
# Criamos o scaler
scaler = StandardScaler()

# Fitamos o scaler APENAS nos dados de TREINO
X_train_scaled = scaler.fit_transform(X_train)

# Aplicamos a transformação (com a média e desvio do treino) nos dados de TESTE
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Dicionário para armazenar nossos modelos
models = {}

# 1. Regressão Logística
print("Treinando Regressão Logística...")
models['Regressão Logística'] = LogisticRegression(random_state=42)
models['Regressão Logística'].fit(X_train_scaled, y_train)

# 2. K-Nearest Neighbors (KNN)
# Vamos começar com k=5 como um palpite inicial
print("Treinando KNN (k=5)...")
models['KNN'] = KNeighborsClassifier(n_neighbors=5)
models['KNN'].fit(X_train_scaled, y_train)

# 3. Árvore de Decisão
print("Treinando Árvore de Decisão...")
models['Árvore de Decisão'] = DecisionTreeClassifier(random_state=42)
models['Árvore de Decisão'].fit(X_train_scaled, y_train)

print("\nModelos treinados com sucesso!")

In [ ]:
# Dicionário para armazenar as acurácias
accuracy_scores = {}

print("--- Comparação de Acurácia (no conjunto de teste) ---")

for model_name, model in models.items():
    # Fazer predições no teste
    y_pred = model.predict(X_test_scaled)

    # Calcular acurácia
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores[model_name] = acc

    print(f"{model_name}: {acc * 100:.2f}%")

# Criar um gráfico de barras simples para comparar
plt.figure(figsize=(10, 5))
plt.bar(accuracy_scores.keys(), accuracy_scores.values())
plt.title('Acurácia dos Modelos')
plt.ylabel('Acurácia')
plt.ylim(0.7, 1.0) # Ajuste o limite se necessário
plt.show()

In [ ]:
# Pegar as predições da Regressão Logística
y_pred_log = models['Regressão Logística'].predict(X_test_scaled)

# Gerar a matriz de confusão
cm = confusion_matrix(y_test, y_pred_log)

# Visualizar a matriz
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Saudável (0)', 'Doente (1)'],
            yticklabels=['Saudável (0)', 'Doente (1)'])
plt.xlabel('Predito pelo Modelo')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão - Regressão Logística')
plt.show()